## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

In [2]:
%%time
import pandas as pd
fi=pd.read_csv('./searches.csv.bz2', sep="^", 
               low_memory=False, dtype=str, chunksize=1000000)
all_chunks=pd.DataFrame()
for i, chunk in enumerate(fi):
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print (i, len(all_chunks))
    
all_chunks.to_csv('searches.no_dup.csv', sep="^", index=False)

0 358999
1 359003
2 359003
3 359003
4 359003
5 359003
6 359003
7 359003
8 359003
9 359003
10 359003
11 359003
12 359003
13 359003
14 359003
15 359003
16 359003
17 359003
18 359003
19 359003
20 359004
CPU times: user 16min 56s, sys: 39 s, total: 17min 35s
Wall time: 17min 15s


In [3]:
%%time
import pandas as pd
fi=pd.read_csv('./bookings.csv.bz2', sep="^", 
               low_memory=False, dtype=str, chunksize=1000000)
all_chunks=pd.DataFrame()
for i, chunk in enumerate(fi):
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print (i, len(all_chunks))
    
all_chunks.to_csv('bookings.no_dup.csv', sep="^", index=False)

0 1000000
1 1000000
2 1000000
3 1000000
4 1000000
5 1000003
6 1000003
7 1000003
8 1000003
9 1000003
10 1000003
CPU times: user 12min 33s, sys: 25.8 s, total: 12min 59s
Wall time: 12min 49s


In [ ]:
#Action plan
1) seleccionar las columnas de bookings 
    (dep_port, arr_port, 'cre_date           ')
2) arreglar espacios en aeropuertos de bookings
3) drop duplicates de bookings
4) poner fecha en mismo formato (str)
4) crear tabla con todo 1
5) search left join con booking
6) fillna(0)
7) save file


In [4]:
b=pd.read_csv('bookings.no_dup.csv', sep="^", 
              usecols=['dep_port', 'arr_port', 'cre_date           '])
s=pd.read_csv('searches.no_dup.csv', sep="^", low_memory=False)

In [5]:
b.columns=b.columns.str.strip()
b.dep_port=b.dep_port.str.strip()
b.arr_port=b.arr_port.str.strip()

In [6]:
b.dep_port[0], b.arr_port[0]

('ZRH', 'LHR')

In [7]:
b.drop_duplicates(inplace=True)

In [8]:
b.cre_date[0]

'2013-02-22 00:00:00'

In [9]:
s.Date[0]

'2013-01-01'

In [10]:
b.cre_date=b.cre_date.str[:10]

In [11]:
b.cre_date[0]

'2013-02-22'

In [12]:
b['booked']=1

In [13]:
s_b=s.merge(b,
           how='left',
           left_on=['Date', 'Origin', 'Destination'],
           right_on=['cre_date', 'dep_port', 'arr_port'])

In [14]:
len(s), len(s_b)

(359004, 359004)

In [15]:
s_b.shape

(359004, 49)

In [16]:
s_b.drop(['cre_date', 'dep_port', 'arr_port'],axis=1, inplace=True)

In [17]:
s_b.shape

(359004, 46)

In [19]:
s_b[ s_b['booked']==1].head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,booked
27,2013-01-01,18:33:28,CCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,JED,2013-01-04,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,1.0
40,2013-01-01,06:36:57,FFP,86f167b84e77346849f9439ae87c02a6,SA,DMM,MNL,1.0,2.0,DMM,MNL,2013-06-01,OJ,NaN,MNL,DMM,2013-06-30,OJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,1.0
59,2013-01-01,07:00:38,FQD,e8741eaf2fa2f71f931475d18fa72096,US,ATL,MIA,0.0,1.0,ATL,MIA,2013-01-10,NV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SEA,1.0
134,2013-01-01,23:30:44,MPT,5215502d9524c3183f3839b0d9a5e4f9,AU,MEL,SYD,0.0,1.0,MEL,SYD,2013-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SYD,1.0
172,2013-01-01,18:34:27,MPT,fd4afff0035bec8f8e305d38804c33f6,IN,BOM,JED,1.0,2.0,BOM,JED,2013-01-26,NaN,NaN,JED,BOM,2013-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,DEL,1.0


In [20]:
s_b['booked']=s_b['booked'].fillna(0)

In [21]:
s_b=s_b.astype({'booked':int})

In [22]:
s_b.to_csv('searches_with_bookings.csv', sep="^", index=False)

In [23]:
!head ./searches_with_bookings.csv

Date^Time^TxnCode^OfficeID^Country^Origin^Destination^RoundTrip^NbSegments^Seg1Departure^Seg1Arrival^Seg1Date^Seg1Carrier^Seg1BookingCode^Seg2Departure^Seg2Arrival^Seg2Date^Seg2Carrier^Seg2BookingCode^Seg3Departure^Seg3Arrival^Seg3Date^Seg3Carrier^Seg3BookingCode^Seg4Departure^Seg4Arrival^Seg4Date^Seg4Carrier^Seg4BookingCode^Seg5Departure^Seg5Arrival^Seg5Date^Seg5Carrier^Seg5BookingCode^Seg6Departure^Seg6Arrival^Seg6Date^Seg6Carrier^Seg6BookingCode^From^IsPublishedForNeg^IsFromInternet^IsFromVista^TerminalID^InternetOffice^booked
2013-01-01^20:25:57^MPT^624d8c3ac0b3a7ca03e3c167e0f48327^DE^TXL^AUH^1.0^2.0^TXL^AUH^2013-01-26^D2^^AUH^TXL^2013-02-02^D2^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^FRA^0
2013-01-01^10:15:33^MPT^b0af35b31588dc4ab06d5cf2986e8e02^MD^ATH^MIL^0.0^1.0^ATH^MIL^2013-01-04^^^^^^^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^KIV^0
2013-01-01^18:04:49^MPT^3561a60621de06ab1badc8ca55699ef3^US^ICT^SFO^1.0^2.0^ICT^SFO^2013-08-02^^